In [1]:
!pip install -q -U \
    langchain \
    langchain-community \
    langchain-core \
    langchain-openai \
    openai \
    newspaper3k \
    trafilatura \
    serpapi

In [2]:
import os
import requests
import trafilatura
from newspaper import Article
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema.agent import AgentAction, AgentFinish
from langchain.agents import AgentExecutor
from functools import partial
from IPython.display import Markdown, display

In [3]:
OPENAI_API_KEY = "your OpenAI API key"
SERPAPI_API_KEY = "your SerpAPI API key"

In [4]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=OPENAI_API_KEY
)

In [5]:
def web_search(query):
    geo = requests.get("https://ipinfo.io").json()
    location = f"{geo.get('city', '')}, {geo.get('region', '')}"
    loc_data = requests.get("https://serpapi.com/locations.json", params={"q": location}).json()
    canonical_location = loc_data[0]["canonical_name"] if loc_data else "United States"

    params = {
        "engine": "google",
        "q": query,
        "location": canonical_location,
        "api_key": SERPAPI_API_KEY
    }
    response = requests.get("https://serpapi.com/search", params=params).json()
    return response

def extract_with_newspaper(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text.strip()
    except:
        return ""

def extract_with_trafilatura(url):
    try:
        downloaded = trafilatura.fetch_url(url)
        return trafilatura.extract(downloaded)
    except:
        return ""

def extract_web_content(response, n_sources=2):
    texts = []
    sources = []
    for i, content in enumerate(response.get("organic_results", [])):
        try:
            title = content.get("title", "")
            url = content.get("link", "")
            snippet = content.get("snippet", "")
            text = (
                extract_with_newspaper(url) or
                extract_with_trafilatura(url) or
                snippet
            )
            if text:
                texts.append(text[:1000])
                sources.append((title, url))
        except:
            continue
        if len(sources) == n_sources:
            break
    return texts, sources

def summarize_text(response, query):
    texts, sources = extract_web_content(response)
    combined = "\n\n".join(texts)
    prompt = (
        "Summarize the following text to answer the user's query.\n"
        "Respond strictly with: Final Answer: <your response>\n\n"
        f"Query: {query}\n\n"
        f"Text:\n{combined}"
    )
    result = llm.invoke(prompt)
    content = result.content.strip().replace("Final Answer: ", "")
    return {
        "summary": content,
        "sources": sources  # list of (title, url)
    }


def related_questions(response, n=2):
    questions = []
    for content in response.get("related_questions", []):
        question = content.get("question", "")
        url = content.get("link", "")
        if question:
            questions.append((question, url))
        if len(questions) == n:
            break
    return questions

def local_map(response):
    try:
        return response["local_map"]["image"]
    except:
        return None

def top_stories(response, n=2):
    stories = []
    for content in response.get("top_stories", []):
        title = content.get("title")
        url = content.get("link", "")
        if title:
            stories.append((title, url))
        if len(stories) == n:
            break
    return stories

def nearby_places(response, n=5):
    try:
        return [place.get("title") for place in response.get("local_results", {}).get("places", [])][:n]
    except:
        return []

def extract_event_results(response):
    output = []

    # 1. Top stories
    top = response.get("top_stories", [])
    if top:
        output.append("**Top Stories:**")
        for story in top[:3]:
            title = story.get("title", "")
            link = story.get("link", "")
            if title:
                if link:
                    output.append(f"- [{title}]({link})")
                else:
                    output.append(f"- {title}")

    # 2. Structured game results
    games = response.get("sports_results", {}).get("games", [])
    if games:
        output.append("\n**Recent Game Results:**")
        for game in games[:3]:
            t1, t2 = game["teams"]
            scoreline = f'{t1["name"]} {t1["score"]} - {t2["score"]} {t2["name"]}'
            venue = game.get("venue", "")
            date = game.get("date", "")
            line = f"- {scoreline} -- {date} at {venue}"
            output.append(line)

    # 3. Organic match info (unstructured)
    matches = []
    for result in response.get("organic_results", []):
        snippet = result.get("snippet", "").lower()
        title = result.get("title", "")
        if any(word in snippet for word in ["score", "defeated", "beat", "match", "draw", "goal", "win", "standings"]):
            matches.append(f"- {title}: {snippet}")

    if matches:
        output.append("\n**Other Match Headlines:**")
        output.extend(matches[:3])


    return "\n".join(output) if output else "No recent results or standings found."



def extract_knowledge_graph(response):
    kg = response.get("knowledge_graph", {})
    if not kg:
        return "No structured information found."

    info = []
    preferred_keys = [
        "title", "type", "description",
        # People
        "born", "died", "spouse", "children", "education", "notable_works", "awards", "website", "image",
        # Organizations
        "founded", "founders", "ceo", "headquarters", "revenue", "subsidiaries",
        # Locations
        "population", "area", "currency", "capital", "official_language",
        # Media
        "release_date", "author", "director", "genre", "starring", "box_office", "duration"
    ]
    
    for key in preferred_keys:
        value = kg.get(key)
        if value:
            if isinstance(value, list):
                value = ", ".join(value)
            info.append(f"**{key.replace('_', ' ').title()}**: {value}")

    # Catch any additional dynamic keys
    for key, value in kg.items():
        if key not in ["title", "type", "description", "born", "died", "spouse", "children", "education", "notable_works", "awards", "website"]:
            if isinstance(value, (str, int, float)) and value:
                info.append(f"**{key.replace('_', ' ').title()}**: {value}")

    return "\n".join(info) if info else "No structured knowledge graph info found."



def classify_query(query):
    prompt = (
        "Classify the query into one of the following categories: "
        "[news, local_place, event, generic].\n"
        "Query: " + query +
        "\nCategory:"
    )
    result = llm.invoke([HumanMessage(content=prompt)])
    return result.content.lower().strip()


def tool_web_search():
    return Tool(
        name="Web Search",
        func=web_search,
        description="Use to search the web based on a user query and geo location"
    )


# Wrapped versions to handle consistent arity
def summarize_text_fixed(response, query):
    return summarize_text(response, query)

def related_questions_fixed(response, _=None):
    return related_questions(response)

def top_stories_fixed(response, _=None):
    return top_stories(response)

def local_map_fixed(response, _=None):
    return local_map(response)

def nearby_places_fixed(response, _=None):
    return nearby_places(response)

def extract_event_results_fixed(response, _=None):
    return extract_event_results(response)

def extract_knowledge_graph_fixed(response, _=None):
    return extract_knowledge_graph(response)

# Tool definitions using shared search_result
def tool_summarize_text(search_result, query):
    return Tool(
        name="Web Summarizer",
        func=partial(summarize_text_fixed, search_result),
        description="Use to summarize web content"
    )

def tool_related_questions(search_result):
    return Tool(
        name="Related Questions",
        func=partial(related_questions_fixed, search_result),
        description="Use to find related or follow-up questions"
    )

def tool_top_stories(search_result):
    return Tool(
        name="Top Stories",
        func=partial(top_stories_fixed, search_result),
        description="Use to extract recent news articles if query is news-related"
    )

def tool_local_map_image(search_result):
    return Tool(
        name="Local Map",
        func=partial(local_map_fixed, search_result),
        description="Use to retrieve map image if the query is about a place or business"
    )

def tool_nearby_places(search_result):
    return Tool(
        name="Nearby Places",
        func=partial(nearby_places_fixed, search_result),
        description="Use to list nearby shops or landmarks"
    )

def tool_event_info(search_result):
    return Tool(
        name="Event Results",
        func=partial(extract_event_results_fixed, search_result),
        description="Use this to extract match results, standings, or breaking news for events like sports"
    )

def tool_knowledge_graph(search_result):
    return Tool(
        name="Knowledge Graph",
        func=partial(extract_knowledge_graph_fixed, search_result),
        description="Use to extract structured facts from the knowledge graph, like birthdate, profession, spouse, and more."
    )

In [6]:
class CaptureHandler(BaseCallbackHandler):
    def __init__(self):
        self.steps = []
        self.final_answer = None

    def on_agent_action(self, action: AgentAction, **kwargs):
        self.steps.append({"tool": action.tool, "input": action.tool_input})

    def on_tool_end(self, output, **kwargs):
        if isinstance(output, dict) and "summary" in output and "sources" in output:
            self.steps[-1]["output"] = output["summary"]
            self.steps[-1]["sources"] = output["sources"]
        elif isinstance(output, str):
            self.steps[-1]["output"] = output.strip()
        elif isinstance(output, list):
            formatted = []
            for x in output:
                if isinstance(x, tuple) and len(x) == 2:
                    title, link = x
                    if title and link and title.strip() and link.startswith("http"):
                        formatted.append(f"- [{title.strip()}]({link})")
                    else:
                        formatted.append(f"- {title or link}")
                elif isinstance(x, str):
                    formatted.append(f"- {x}")
                else:
                    formatted.append(f"- {str(x)}")
            self.steps[-1]["output"] = "\n".join(formatted)
        else:
            self.steps[-1]["output"] = str(output)

    def on_agent_finish(self, finish: AgentFinish, **kwargs):
        self.final_answer = finish.return_values.get("output", None)

In [7]:
def format_agent_output(handler: CaptureHandler):
    section_map = {
        "web summarizer": ("Summary", 2),
        "top stories": ("Top Stories", 3),
        "event results": ("Event Highlights", 3),
        "local map": ("Local Map", 3),
        "nearby places": ("Nearby Places", 3),
        "related questions": ("Related Questions", 4),
        "knowledge graph": ("Facts & Overview", 3),
    }
    skip_display_tools = {"knowledge graph", "local map", "nearby places", "web summarizer"}
    sections_by_priority = {1: [], 2: [], 3: [], 4: []}
    seen_tools = set()

    # Organize tool outputs by display priority
    for step in handler.steps:
        tool = step["tool"].lower()
        if tool in skip_display_tools:
            continue
        output = step.get("output", "").strip()
        if not output or tool in seen_tools:
            continue

        title, priority = section_map.get(tool, (f"{step['tool']}", 3))
        sections_by_priority[priority].append(f"### {title}\n{output}")
        seen_tools.add(tool)

    # Final Answer first
    if handler.final_answer:
        answer = handler.final_answer.replace("Final Answer: ", "").strip()
        sources_md = ""
        for step in handler.steps:
            if step["tool"].lower() == "web summarizer" and step.get("sources"):
                sources_md = "\n\n**Sources:**\n" + "\n".join(
                    f"- [{title}]({url})" for title, url in step["sources"]
                )
                break
        for step in handler.steps:
            if step["tool"].lower() == "local map":
                map_image_url = step.get("output", "").strip()
                if map_image_url and map_image_url.startswith("http"):
                    answer += f"\n\n![Map]({map_image_url})"
                break
        sections_by_priority[1].insert(0, f"### Final Answer\n{answer}{sources_md}")


    # Concatenate in priority order
    ordered_output = "\n\n---\n\n".join(
        section for priority in [1, 2, 3, 4]
        for section in sections_by_priority[priority]
    )

    # Display in Jupyter
    display(Markdown(ordered_output))

In [8]:
def run_agent_with_classification(query):
    category = classify_query(query)
    if category in ("event", "news"):
        query = query + " today"

    search_result = web_search(query)

    selected_tools = [
        tool_summarize_text(search_result, query),
        tool_top_stories(search_result),
        tool_local_map_image(search_result),
        tool_nearby_places(search_result),
        tool_event_info(search_result),
        tool_related_questions(search_result),
        tool_knowledge_graph(search_result)
    ]
    
    agent = initialize_agent(
        tools=selected_tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=10,
        agent_kwargs={
            "prefix": (
                "You are a helpful search assistant. Your task is to gather high-quality, relevant information using web tools.\n\n"
                "You MUST use the following tools:\n"
                "- Web Summarizer: Always use this first unless the query clearly depends on a map or place list.\n"
                "- One or more contextual tools based on the query category (Top Stories, Event Results, Nearby Places, Local Map, Knowledge Graph):\n"
                "  - If the query is related to geo-location, you MUST use both Local Map and Nearby Places.\n"
                "  - If you use Nearby Places, you MUST summarize each place using Web Summarizer.\n"
                "  - If the query is about a person, use Knowledge Graph.\n"
                "  - If any tool provides an image, only show it in the Final Answer.\n"
                "- Related Questions: Always use this before Final Answer.\n\n"

                "⚠️ IMPORTANT RULES ⚠️\n"
                "- NEVER display markdown, links, images, or bullet points until you reach the Final Answer step.\n"
                "- NEVER write a complete paragraph or summary before the Final Answer.\n"
                "- ONLY one Action per step. Do not merge tool calls.\n"
                "- Do not skip Web Summarizer (unless using map/places first) or Related Questions.\n"
                "- Do NOT output image URLs outside Final Answer.\n\n"

                "Follow this format strictly for every query:\n\n"

                "Thought: what you want to do and which tool to use\n"
                "Action: tool name\n"
                "Action Input: the input to the tool\n"
                "Observation: the result\n\n"

                "Thought: which contextual tool to use next (Top Stories, Event Results, Nearby Places, Local Map, Knowledge Graph)?\n"
                "Action: tool name\n"
                "Action Input: the input to the tool\n"
                "Observation: the result\n\n"

                "Repeat the above if needed. Only one Action per Thought.\n\n"

                "Thought: use the Related Questions tool before finishing\n"
                "Action: Related Questions\n"
                "Action Input: the input\n"
                "Observation: the result\n\n"

                "Thought: conclude your reasoning\n"
                "Final Answer: <your full answer here using markdown, links, and image URLs>\n"
            )
        }
    )

    handler = CaptureHandler()
    executor = AgentExecutor.from_agent_and_tools(agent.agent, tools=selected_tools, verbose=True)
    executor.invoke({"input": query}, config={"callbacks": [handler]})
    
    output_md = format_agent_output(handler)

In [10]:
# Example Use 1
run_agent_with_classification("coffee near me")



> Entering new AgentExecutor chain...
Thought: The query is related to geo-location, so I should use the Local Map tool to find coffee places nearby.
Action: Local Map
Action Input: coffee near me
Observation: https://serpapi.com/searches/685e449da3f7300caf71abcd/images/a9ef037d8f7d983ba41a4d130b478abd.png
Thought:The query is related to geo-location, so I should use the Nearby Places tool to list coffee shops nearby. After that, I will summarize each place using the Web Summarizer.

Action: Nearby Places
Action Input: coffee near me
Observation: ['Capture Coffee', 'Philz Coffee', 'Qamaria Yemeni Coffee Co.']
Thought:I should use the Web Summarizer to gather more information about each of the coffee places listed: Capture Coffee, Philz Coffee, and Qamaria Yemeni Coffee Co.

Action: Web Summarizer
Action Input: Capture Coffee
Observation: {'summary': 'Qamaria Yemeni Coffee is a late-night coffee shop in Fremont, California.', 'sources': [('Late Night Coffee Shops Fremont, CA - Yelp', 

### Final Answer
Here are some coffee shops near you:

1. **Capture Coffee**: A coffee shop located in Fremont, California, known for its late-night service. [More info](https://www.yelp.com/search?find_desc=Late+Night+Coffee+Shops&find_loc=Fremont%2C+CA)

2. **Philz Coffee**: Unfortunately, there is no specific information available about Philz Coffee from the sources.

3. **Qamaria Yemeni Coffee Co.**: Another late-night coffee shop located in Fremont, California. [More info](https://www.yelp.com/search?find_desc=Late+Night+Coffee+Shops&find_loc=Fremont%2C+CA)

You can view the map of these locations [here](https://serpapi.com/searches/685e449da3f7300caf71abcd/images/a9ef037d8f7d983ba41a4d130b478abd.png).

![Map](https://serpapi.com/searches/685e449da3f7300caf71abcd/images/a9ef037d8f7d983ba41a4d130b478abd.png)

**Sources:**
- [Late Night Coffee Shops Fremont, CA - Yelp](https://www.yelp.com/search?find_desc=Late+Night+Coffee+Shops&find_loc=Fremont%2C+CA)
- [THE 10 BEST Cafés in Fremont (Updated 2025)](https://www.tripadvisor.com/Restaurants-g32411-c8-Fremont_California.html)

In [11]:
# Example Use 2
run_agent_with_classification("fifa club world cup")



> Entering new AgentExecutor chain...
Thought: The query is about an event, specifically the FIFA Club World Cup. I should use the Web Summarizer to gather information about today's matches or events related to the FIFA Club World Cup.
Action: Web Summarizer
Action Input: FIFA Club World Cup today
Observation: {'summary': "The text does not provide specific information about today's FIFA Club World Cup matches or events.", 'sources': [('Scores & Fixtures', 'https://www.fifa.com/en/tournaments/mens/club-world-cup/usa-2025/scores-and-fixtures'), ('FIFA Club World Cup Schedule', 'https://www.espn.com/soccer/schedule/_/league/fifa.cwc')]}
Thought:The Web Summarizer did not provide specific information about today's matches. I should use the Event Results tool to get the latest match results or schedule for the FIFA Club World Cup.

Action: Event Results
Action Input: FIFA Club World Cup
Observation: **Top Stories:**
- [Club World Cup bracket: Schedule, scores, and results for FIFA soccer

### Final Answer
Recent results from the FIFA Club World Cup include Al-Hilal defeating Pachuca 2-0, Real Madrid winning against RB Salzburg 3-0, and Al Ain beating Wydad AC 2-1. For more details on upcoming matches or today's schedule, you can check the official FIFA website or sports news outlets like ESPN or CBS Sports. Here are some useful links for more information: [FIFA Club World Cup Schedule](https://www.fifa.com/en/tournaments/mens/club-world-cup/usa-2025/scores-and-fixtures), [ESPN Soccer Schedule](https://www.espn.com/soccer/schedule/_/league/fifa.cwc).

**Sources:**
- [Scores & Fixtures](https://www.fifa.com/en/tournaments/mens/club-world-cup/usa-2025/scores-and-fixtures)
- [FIFA Club World Cup Schedule](https://www.espn.com/soccer/schedule/_/league/fifa.cwc)

---

### Event Highlights
**Top Stories:**
- [Club World Cup bracket: Schedule, scores, and results for FIFA soccer tournament](https://www.usatoday.com/story/sports/soccer/tourneys/2025/06/26/club-world-cup-bracket-schedule-scores-pairings/84376551007/)
- [FIFA Club World Cup 2025 schedule, scores: Where to watch Real Madrid vs. RB Salzburg in group stage](https://www.cbssports.com/soccer/news/fifa-club-world-cup-2025-schedule-scores-where-to-watch-real-madrid-vs-rb-salzburg-in-group-stage/)
- [Juventus FC vs Manchester City Highlights | FIFA Club World Cup 2025](https://www.youtube.com/watch?v=P-XF0rdWm_s)

**Recent Game Results:**
- Al-Hilal 2 - 0 Pachuca -- Yesterday at Geodis Park
- RB Salzburg 0 - 3 Real Madrid -- Yesterday at Lincoln Financial Field
- Wydad AC 1 - 2 Al Ain -- Yesterday at Audi Field

**Other Match Headlines:**
- Scores & Fixtures: follow the fifa club world cup 2025™ with live scores, results and upcoming fixtures.
- FIFA Club World Cup 2025™: view the full fixture schedule for the new-look tournament in the usa, with dates, kick-off times and venues for all 63 matches.
- View the Club World Cup 2025 fixtures and results: view all the fixtures and results for the new tournament in the usa, with dates, kick-off times and venues for all 63 matches.

---

### Related Questions
- [Where can I watch Real Madrid vs RB Salzburg?](https://www.dazn.com/en-GB/news/football/red-bull-salzburg-real-madridpreview-fifa-club-world-cup-match-how-to-watch-cwc25uk/w5gqgt0f1hwu1p6tpz6gbrjnf)
- [How much does the Club World Cup winner get?](https://www.standard.co.uk/sport/football/chelsea-fc-prize-money-club-world-cup-b1234843.html#:~:text=In%20total%2C%20FIFA%20are%20awarding,they%20win%20all%20their%20matches.)

In [12]:
# Example Use 3
run_agent_with_classification("global inflation trends 2025")



> Entering new AgentExecutor chain...
I want to gather information on global inflation trends for 2025. I will start by using the Web Summarizer to get an overview of the topic. 
Action: Web Summarizer
Action Input: global inflation trends 2025
Observation: {'summary': 'Average headline inflation across the OECD is projected to be 4.2% in 2025.', 'sources': [('World Economic Outlook (April 2025) - Inflation rate, ...', 'https://www.imf.org/external/datamapper/PCPIPCH@WEO/WEOWORLD'), ('Visualized: Global Inflation Projections in 2025 and 2026', 'https://www.visualcapitalist.com/visualized-global-inflation-projections-in-2025-and-2026/')]}
Thought:I have gathered some information on global inflation trends for 2025. Next, I will use the Related Questions tool to find any additional relevant questions or information on this topic.

Action: Related Questions
Action Input: global inflation trends 2025
Observation: [('What is the global inflation rate in 2025?', 'https://www.worldbank.org/

### Final Answer
Global inflation trends for 2025 indicate that the average headline inflation across the OECD is projected to be 4.2%. For more detailed insights, you can refer to the [World Economic Outlook](https://www.imf.org/external/datamapper/PCPIPCH@WEO/WEOWORLD) and [Visual Capitalist's projections](https://www.visualcapitalist.com/visualized-global-inflation-projections-in-2025-and-2026/). Additionally, related questions such as "What is the global inflation rate in 2025?" can be explored through resources like the [World Bank's press release](https://www.worldbank.org/en/news/press-release/2025/06/10/global-economic-prospects-june-2025-press-release).

**Sources:**
- [World Economic Outlook (April 2025) - Inflation rate, ...](https://www.imf.org/external/datamapper/PCPIPCH@WEO/WEOWORLD)
- [Visualized: Global Inflation Projections in 2025 and 2026](https://www.visualcapitalist.com/visualized-global-inflation-projections-in-2025-and-2026/)

---

### Related Questions
- [What is the global inflation rate in 2025?](https://www.worldbank.org/en/news/press-release/2025/06/10/global-economic-prospects-june-2025-press-release)
- [What are the inflation expectations for 2025?](https://www.google.com/search?sca_esv=9df8d0d36b3fc424&q=Cleveland+Fed%27s+&sa=X&ved=2ahUKEwjR4ZmOh5GOAxVhg4kEHVOiHWkQxccNegUI7wEQAQ)

In [13]:
# Example Use 4
run_agent_with_classification("urgent care san francisco")



> Entering new AgentExecutor chain...
Thought: The query is related to geo-location, so I should use the Local Map tool to retrieve a map image of urgent care locations in San Francisco.
Action: Local Map
Action Input: urgent care San Francisco
Observation: https://serpapi.com/searches/685e45466c4c0c07c2ff887d/images/5f0a720783abb501fcaf4c592f747f86.png
Thought:Thought: I should use the Nearby Places tool to list nearby urgent care centers in San Francisco.
Action: Nearby Places
Action Input: urgent care San Francisco
Observation: ['Dore Urgent Care Clinic (Psychiatric)', 'UCSF Bayfront Urgent Care', 'UCSF Emergency Department']
Thought:Thought: I should use the Web Summarizer to gather more information about each of the listed urgent care centers in San Francisco.
Action: Web Summarizer
Action Input: Dore Urgent Care Clinic (Psychiatric) San Francisco
Observation: {'summary': 'The text does not provide specific information about the Dore Urgent Care Clinic (Psychiatric) in San Franc

### Final Answer
If you're looking for urgent care in San Francisco, here are some options:

1. **Dore Urgent Care Clinic (Psychiatric)**: Specific information about this clinic is not readily available, but it is mentioned in the context of urgent care services in the area.

2. **UCSF Bayfront Urgent Care**: This facility is part of a partnership between UCSF Health and GoHealth, offering urgent care services at multiple Bay Area locations, including pediatric care. It is staffed by UCSF Health Community Medical Group providers.

3. **UCSF Emergency Department**: While specific details about the emergency department are not provided, UCSF Health collaborates with GoHealth to offer urgent care services, including pediatric and after-hours care.

For a visual representation, you can view the map of urgent care locations in San Francisco [here](https://serpapi.com/searches/685e45466c4c0c07c2ff887d/images/5f0a720783abb501fcaf4c592f747f86.png).

Additionally, if you're concerned about costs, you might want to explore how much urgent care costs without insurance in San Francisco or compare the costs with emergency room visits. More information on these topics can be found [here](https://enhancehealth.com/blog/how-much-does-urgent-care-cost-without-insurance/#:~:text=For%20example%2C%20the%20average%20cost,in%20Cincinnati%20OH%20is%20%24150.) and [here](https://support.google.com/websearch?p=ai_overviews&hl=en-CA).

![Map](https://serpapi.com/searches/685e45466c4c0c07c2ff887d/images/5f0a720783abb501fcaf4c592f747f86.png)

**Sources:**
- [GoHealth Urgent Care](https://www.ucsfhealth.org/about/our-affiliates/gohealth-urgent-care)
- [Get urgent care services](https://www.sf.gov/get-urgent-care-services)

---

### Related Questions
- [How much is urgent care without insurance in San Francisco?](https://enhancehealth.com/blog/how-much-does-urgent-care-cost-without-insurance/#:~:text=For%20example%2C%20the%20average%20cost,in%20Cincinnati%20OH%20is%20%24150.)
- [Is urgent care cheaper than ER?](https://support.google.com/websearch?p=ai_overviews&hl=en-CA)